<a href="https://colab.research.google.com/github/advcloud/colabtest/blob/main/wandbt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{keras-video} -->

# W&B 💘 Keras
Use Weights & Biases for machine learning experiment tracking, dataset versioning, and project collaboration.

<div><img /></div>

<img src="https://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />

<div><img /></div>


# Install, Import, and Log In

In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

### Step 0: Install W&B

In [2]:
%%capture
!pip install wandb

### Step 1: Import W&B and Login

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

> Side note: If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login` will take you to sign-up/login page. Signing up is easy!

# Download and Prepare the Dataset


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train[::5] / 255., x_test / 255.
y_train = y_train[::5]

CLASS_NAMES = ["airplane", "automobile", "bird", "cat",
               "deer", "dog", "frog", "horse", "ship", "truck"]

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

170508288/170498071 [==============================] - 2s 0us/step
Shape of x_train:  (10000, 32, 32, 3)
Shape of y_train:  (10000, 1)
Shape of x_test:  (10000, 32, 32, 3)
Shape of y_test:  (10000, 1)


# Define the Model

Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [5]:
def Model():
  inputs = keras.layers.Input(shape=(32, 32, 3))

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=2)(x)

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  
  outputs = keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

# Train the Model

### Step 2: Give `wandb.init` your `config`

You first initialize your wandb run, letting us know some training is about to happen. [Check out the official documentation for `.init` here $\rightarrow$](https://docs.wandb.com/library/init)

That's when you need to set your hyperparameters.
They're passed in as a dictionary via the `config` argument,
and then become available as the `config` attribute of `wandb`.

Learn more about `config` in this [Colab Notebook $\rightarrow$](http://wandb.me/config-colab)

In [6]:
# Initialize wandb with your project name
run = wandb.init(project='my-keras-integration',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.005,
                     "epochs": 5,
                     "batch_size": 1024,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "CIFAR-10"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
# Notice that we use config, so our metadata matches what gets executed
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

wandb: Currently logged in as: clair (claireye). Use `wandb login --relogin` to force relogin


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 32)        9248      
                                                             

### Step 3: Pass `WandbCallback` to `model.fit`

Keras has a [robust callbacks system](https://keras.io/api/callbacks/) that
allows users to separate model definition and the core training logic
from other behaviors that occur during training and testing.

That includes, for example, 

**Click on the Project page link above to see your results!**

In [7]:
# We train with our beloved model.fit
# Notice WandbCallback is used as a regular callback
# We again use config
_ = model.fit(x_train, y_train,
          epochs=config.epochs, 
          batch_size=config.batch_size,
          validation_data=(x_test, y_test),
          callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5
10/10 [==============================] - 31s 3s/step - loss: 2.2907 - acc: 0.1112 - val_loss: 2.2962 - val_acc: 0.1017 - _timestamp: 1653640452.0000 - _runtime: 292.0000
Epoch 2/5
10/10 [==============================] - 28s 3s/step - loss: 2.2554 - acc: 0.1357 - val_loss: 2.1233 - val_acc: 0.1683 - _timestamp: 1653640480.0000 - _runtime: 320.0000
Epoch 3/5
10/10 [==============================] - 28s 3s/step - loss: 2.1588 - acc: 0.1721 - val_loss: 2.1405 - val_acc: 0.1484 - _timestamp: 1653640509.0000 - _runtime: 349.0000
Epoch 4/5
10/10 [==============================] - 28s 3s/step - loss: 2.1109 - acc: 0.1752 - val_loss: 2.0569 - val_acc: 0.1949 - _timestamp: 1653640537.0000 - _runtime: 377.0000
Epoch 5/5
10/10 [==============================] - 28s 3s/step - loss: 2.0489 - acc: 0.1954 - val_loss: 2.0220 - val_acc: 0.2196 - _timestamp: 1653640565.0000 - _runtime: 405.0000


# Use `wandb.log` for custom metrics

Here, we log the error rate on the test set.

In [8]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test Error Rate: ', round((1 - accuracy) * 100, 2))

# With wandb.log, we can easily pass in metrics as key-value pairs.
wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

run.finish()

313/313 [==============================] - 7s 22ms/step - loss: 2.0220 - acc: 0.2196
Test Error Rate:  78.04


Test Error Rate,▁
acc,▁▃▆▆█
epoch,▁▃▅▆█
loss,█▇▄▃▁
val_acc,▁▅▄▇█
val_loss,█▄▄▂▁
Test Error Rate,78.04
acc,0.1954
best_epoch,4
best_val_loss,2.02201
epoch,4


# Log predictions on test data with `WandbCallback`

In [9]:
# Initialize wandb with your project name
run = wandb.init(project='my-keras-integration',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "CIFAR-10"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 32)        9248      
                                                             

In [10]:
# We focus on a subset of images, since this is for human review
val_images, val_labels = x_test[:32], y_test[:32]

# Our beloved model.fit returns
# By passing arguments to WandbCallback, we change its behavior
_ = model.fit(x_train, y_train,
              epochs=config.epochs, 
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image', 
                                       validation_data=(val_images, val_labels), 
                                       labels=CLASS_NAMES)])

run.finish()

wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/5
313/313 [==============================] - 36s 113ms/step - loss: 2.0887 - acc: 0.1937 - val_loss: 1.8994 - val_acc: 0.2633 - _timestamp: 1653640728.0000 - _runtime: 53.0000
Epoch 2/5
313/313 [==============================] - 35s 111ms/step - loss: 1.8573 - acc: 0.2735 - val_loss: 1.8111 - val_acc: 0.3009 - _timestamp: 1653640763.0000 - _runtime: 88.0000
Epoch 3/5
313/313 [==============================] - 35s 110ms/step - loss: 1.7734 - acc: 0.3138 - val_loss: 1.6803 - val_acc: 0.3579 - _timestamp: 1653640798.0000 - _runtime: 123.0000
Epoch 4/5
313/313 [==============================] - 34s 109ms/step - loss: 1.6694 - acc: 0.3578 - val_loss: 1.6763 - val_acc: 0.3493 - _timestamp: 1653640832.0000 - _runtime: 157.0000
Epoch 5/5
313/313 [==============================] - 35s 111ms/step - loss: 1.5891 - acc: 0.4032 - val_loss: 1.5495 - val_acc: 0.4218 - _timestamp: 1653640867.0000 - _runtime: 192.0000


acc,▁▄▅▆█
epoch,▁▃▅▆█
loss,█▅▄▂▁
val_acc,▁▃▅▅█
val_loss,█▆▄▄▁
acc,0.4032
best_epoch,4
best_val_loss,1.54947
epoch,4
loss,1.58907
val_acc,0.4218
